中文分词是把连续的汉字分隔成具有语言语义学意义的词。现代汉语继承自古代汉语的传统，词与词之间没有分隔符。而词是具有语义的最基本单元，要让计算机理解中文，分词是最基础也是最重要的步骤。分词结果的优劣直接影响词法分析、句法分析、篇章分析等其他自然语言处理任务的效果，是任何中文自然语言处理系统难以回避的第一道工序。

分词算法中首先出现的是基于词典匹配的算法，其中最具代表性的就是最大正向匹配算法、最大逆向匹配算法。刘源，梁南元在1986年首次将最大正向匹配算法应用到汉语自动分词系统中 。梁南元的实验结果显示，在词典完备、没有任何其它知识的条件下，正向最大匹配法的错误切分率为1/169，逆向最大匹配法的错误切分率为1/245，这主要是因为汉语中心词多在词的右部的原因

#### 1. 最大正向匹配算法

最大正向匹配算法（Forward Maximum Matching, FMM）从句子左端开始，不断匹配最长的词（组不了词的单字则单独划开），直到把句子划分完。算法的理由很简单：人在阅读时也是从左往右逐字读入的，最大匹配法是与人的习惯相符的。但是该算法有很大的局限性，切分准确率不高。例如，“北京大学生前来应聘”会被误分成“北京大学／生前／来／应聘”。

![img](https://ask.qcloudimg.com/http-save/yehe-1676324/gv5ns0jh2h.jpeg?imageView2/2/w/1620)

#### 2. 最大逆向匹配算法

最大逆向匹配算法（Reverser Maximum Matching, RMM）从句子右端开始，不断匹配最长的词（组不了词的单字则单独划开），直到把句子划分完。该算法切分效果比最大正向匹配效果好一点，但是同样会造成不少切分错误。例如，“长春药店”会被误分为“长/春药店”。

#### 3. 最大双向匹配算法

正向最大匹配法和逆向最大匹配法，都有其局限性,因此有人又提出了双向最大匹配法（Bi-directional Maximum Matching, BMM）。双向最大匹配法，即两种算法都切一遍，然后根据大颗粒度词越多越好，非词典词和单字词越少越好等规则，选取其中一种分词结果输出。孙茂松等人发现汉语文本中90.0％左右的句子，FMM和RMM的切分完全重合且正确，9.0％左右的句子FMM和RMM切分不同，但其中必有一个是正确的

双向最大匹配法是将正向最大匹配法得到的分词结果和逆向最大匹配法的到的结果进行比较，从而决定正确的分词方法。据SunM.S. 和 Benjamin K.T.（1995）的研究表明，中文中90.0％左右的句子，正向最大匹配法和逆向最大匹配法完全重合且正确，只有大概9.0％的句子两种切分方法得到的结果不一样，但其中必有一个是正确的（歧义检测成功），只有不到1.0％的句子，或者正向最大匹配法和逆向最大匹配法的切分虽重合却是错的，或者正向最大匹配法和逆向最大匹配法切分不同但两个都不对（歧义检测失败）。这正是双向最大匹配法在实用中文信息处理系统中得以广泛使用的原因所在。

在本文实现的方法中，是综合考虑了正向和逆向最大匹配的结果，加入了一些启发式的规则来对分词结果进行进一步消歧的。

启发式规则：

    1.如果正反向分词结果词数不同，则取分词数量较少的那个。

    2.如果分词结果词数相同

                 a.分词结果相同，就说明没有歧义，可返回任意一个。

                 b.分词结果不同，返回其中单字较少的那个。


#### 4. 代码实现

In [2]:
class WordSegmentation:
    def __init__(self, dict_path):
        """
        """
        self._dict_path = dict_path 
        
        with open(self._dict_path, "r") as f:
            self._dict_list = f.read().split("\n")[0:-1]
        
        self._max_token_len = max([len(item) for item in self._dict_list])

    def fmm(self, string):
        """正向最大匹配算法
        Forward Maximum Matching, FMM 
        """
        result = []
        while string:  
            sub_str = string[0:self._max_token_len]
            
            while sub_str not in self._dict_list and len(sub_str) != 1:
                sub_str = sub_str[0:-1]
            result.append(sub_str)
            string = string[len(sub_str):]
        return result
        
    def rmm(self, string):
        """逆向最大匹配算法
        Reverser Maximum Matching, RMM
        """
        result = []
        while string:  
            sub_str = string[-self._max_token_len:]
            
            while sub_str not in self._dict_list and len(sub_str) != 1:
                sub_str = sub_str[1:]
            result.append(sub_str)
            string = string[0:-len(sub_str)]
        return result[::-1] 
      
    def bmm(self, string):
        """双向最大匹配算法
        Bi-directional Maximum Matching, BMM
        """
        fmm_result = self.fmm(string)
        rmm_result = self.rmm(string)
        
        # 筛选规则
        # step1: 如果正反向分词结果词数不同，则取分词数量较少的那个。
        if len(fmm_result) > len(rmm_result):
            return rmm_result
        if len(fmm_result) < len(rmm_result):
            return fmm_result 
        # step2: 返回其中单字较少的那个 
        if sum([1 if len(item) == 1 else 0 for item in fmm_result]) > sum([1 if len(item) == 1 else 0 for item in rmm_result]):
            return rmm_result
        if sum([1 if len(item) == 1 else 0 for item in fmm_result]) < sum([1 if len(item) == 1 else 0 for item in rmm_result]):
            return fmm_result 
        
        # 返回任意结果 
        return fmm_result 

In [13]:
ws = WordSegmentation("dic.txt")

# case1 : 北京大学生前来应聘 
print("北京大学生前来应聘")
print("fmm:\t", ws.fmm("北京大学生前来应聘"))
print("rmm:\t", ws.rmm("北京大学生前来应聘"))
print("bmm:\t", ws.bmm("北京大学生前来应聘"))

# case2 : 我一个人吃饭 
print("我一个人吃饭")
print("fmm:\t", ws.fmm("我一个人吃饭"))
print("rmm:\t", ws.rmm("我一个人吃饭"))
print("bmm:\t", ws.bmm("我一个人吃饭"))

# case3: 计算语言学课程是三个课时
print("计算语言学课程是三个课时")
print("fmm:\t", ws.fmm("计算语言学课程是三个课时"))
print("rmm:\t", ws.rmm("计算语言学课程是三个课时"))
print("bmm:\t", ws.bmm("计算语言学课程是三个课时"))

# case4: 我们中出了一个叛徒
print("我们中出了一个叛徒")
print("fmm:\t", ws.fmm("我们中出了一个叛徒"))
print("rmm:\t", ws.rmm("我们中出了一个叛徒"))
print("bmm:\t", ws.bmm("我们中出了一个叛徒"))

北京大学生前来应聘
fmm:	 ['北京大学', '生前', '来', '应聘']
rmm:	 ['北京', '大学生', '前来', '应聘']
bmm:	 ['北京', '大学生', '前来', '应聘']
我一个人吃饭
fmm:	 ['我', '一个', '人', '吃饭']
rmm:	 ['我', '一', '个人', '吃饭']
bmm:	 ['我', '一个', '人', '吃饭']
计算语言学课程是三个课时
fmm:	 ['计算', '语言学', '课程', '是', '三个', '课时']
rmm:	 ['计算', '语言学', '课程', '是', '三个', '课时']
bmm:	 ['计算', '语言学', '课程', '是', '三个', '课时']
我们中出了一个叛徒
fmm:	 ['我们', '中出', '了', '一个', '叛徒']
rmm:	 ['我们', '中出', '了', '一个', '叛徒']
bmm:	 ['我们', '中出', '了', '一个', '叛徒']
